# Notebook for calculating time to detection, cases, hospitalisations and deaths for varying community and hospital testing scenarios

Definitions used in this model.<br>

Infection Hospitalisation Rate (IHR): The percentage of people infected with a disease that are expected to be hospitalised.<br>
Infection Fatality Rate (IFR): The percentage of people infected with a disease that are expected to die.<br>
Infection Detection Rate (IDR): The percentage of people infected with a disease that are expected to be detected (in this simulation either via community testing or upon hospitalisation).

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import statistics
import os
from matplotlib.ticker import MaxNLocator
from scipy.stats import weibull_min, t

# Set global fonts to Aerial
plt.rcParams.update({"font.family": "sans-serif"})
plt.rcParams.update({"font.sans-serif": "Arial"})

### Set seed for reproducable results

In [ ]:
np.random.seed(0)

In [ ]:
def TTD(
    age_specific_IDRs,
    age_specific_IHRs,
    age_specific_IFRs,
    age_pop_distribution,
    R=1.2,
    serial_interval=3,
    testing_regime=str(),
    initial_cases=1,
    testing_rate=1,
    hospital_admission_delay=0,
    ICU_admission_delay=0,
    testing_delay=0,
):
    """
    Calculates the time to detection of an outbreak based on scenario specific parameters.
    Associated cases, hospitalisations and deaths are calculated at the point in time of detection.

    Parameters
    ----------
    age_specific_IDRs: dict
        Age band specific detection rates (%).
        keys:str, age band.
        values:int or float between 0 - 100.

    age_specific_IHRs: dict
        Age band specific hospitalisation rates (%).
        keys: str, age band.
        values: int or float, between 0 - 100.

    age_specific_IFRs: dict
        Age band specific fatality rates (%).
        keys: str, age band.
        values: int or float, between 0 - 100.

    age_pop_distribution: dict
        Age band specific proportion of English population.
        keys: str, age band.
        values: float. Values sum to 1.

    R: float
        Reproduction number.

    serial_interval: int or float
        Random variate from a Weibull distribution.

    testing_regime: str
        Name / label of testing regime.

    initial_cases: int
        Number of initial cases.

    testing_rate: int or float
        Proportion of cases that are tested. The lower the proportion of cases tested, the longer until a positive case is detected.
        i.e. If testing only occurs in ICU, and the ICU rate of hospitalisations is 0.25, 4x as many positive cases will need to admitted
        to hospital until 1 case is detected.

    hospital_admission_delay: int or float
        Number of days (and therefore additional cycles) from infection to hospitalisation.

    ICU_admission_delay: int or float
        Number of days (and therefore additional cycles) from hospitalisation if ICU admission.

    testing_delay: int or float
        Number of days (and therefore additional cycles) from  point of test to test result.

    Returns
    -------
    TTD_breakdown:dict
        Summary of time to detection associated figures. i.e. days, total cases/hospitalisation/deaths and age specific cases/hospitalisation/deaths.
        keys: str, name of figure.
        values: float, associated figure.
    """

    growth_rate = R ** (1 / serial_interval)

    # Initialise values
    new_cases = initial_cases
    total_cases = initial_cases
    detections = sum(
        [
            total_cases * (age_specific_IDRs[band] / 100) * age_pop_distribution[band]
            for band in age_specific_IDRs
        ]
    )

    # Calculate days to detection. Cycle until the sum of age specific IDRs * total cases == (1 / testing_rate).
    # The testing rate indicates the proportion of hospitalisations tested (i.e. when testing occurs in ICU only).
    # If the testing rate is 0.25, then you who need 1/0.25 (4) hospitalisations total before one case is detected (if only one quarter of admissions end in ICU).
    # If the testing rate is 1, all hospitalisations are being tested and therefore keep cycling until the sum of age specific IDRs * total cases == 1.
    days = 1
    while detections < (1 / testing_rate):
        new_cases = new_cases * growth_rate
        total_cases = total_cases + new_cases
        detections = sum(
            [
                total_cases
                * (age_specific_IDRs[band] / 100)
                * age_pop_distribution[band]
                for band in age_specific_IDRs
            ]
        )

        days += 1

    # Keep cycling an additional number of days to account for delay
    for i in range(hospital_admission_delay + ICU_admission_delay + testing_delay):
        new_cases = new_cases * growth_rate
        total_cases = total_cases + new_cases
        detections = sum(
            [
                total_cases
                * (age_specific_IDRs[band] / 100)
                * age_pop_distribution[band]
                for band in age_specific_IDRs
            ]
        )

        days += 1

    # No. cases per age bracket at TTD
    age_specific_cases = {
        "cases_" + band: total_cases * age_pop_distribution[band]
        for band in age_pop_distribution
    }

    # No. hospitalisations per age bracket as TTD
    age_specific_hospitalisations = {
        "hosp_"
        + band: total_cases
        * (age_specific_IHRs[band] / 100)
        * age_pop_distribution[band]
        for band in age_specific_IHRs
    }

    # Total no. hospitalisations
    total_hospitalisations = sum(
        [
            total_cases * (age_specific_IHRs[band] / 100) * age_pop_distribution[band]
            for band in age_specific_IHRs
        ]
    )

    # No. deaths per age bracket at TTD
    age_specific_deaths = {
        "death_"
        + band: total_cases
        * (age_specific_IFRs[band] / 100)
        * age_pop_distribution[band]
        for band in age_specific_IFRs
    }

    # Total no. of deaths
    total_deaths = sum(
        [
            total_cases * (age_specific_IFRs[band] / 100) * age_pop_distribution[band]
            for band in age_specific_IFRs
        ]
    )

    # Summary dictionary of total values
    totals = {
        "TTD": days,
        "total_cases": total_cases,
        "total_hospitalisations": total_hospitalisations,
        "total_deaths": total_deaths,
    }

    # Summary dictionary of total values and age specific breakdowns
    TTD_breakdown = {
        **totals,
        **age_specific_cases,
        **age_specific_hospitalisations,
        **age_specific_deaths,
    }

    return TTD_breakdown


def assess_scenarios(scenarios, R=1.2, shape=3, scale=2.8, num_samples=100000):
    """
    Cycles through a list of scenarios, each with it's own parameters, and simulates them 'num_samples' amount of times.
    The serial is a random variate from the Weibull distribution. The median value for the time to detection, total cases and associated
    figures are calculated and returned.

    Parameters
    ----------
    R: float
        Reproduction number.

    shape: int or float
        Shape parameter of the Weibull distribution.

    scale: int or float
        Scale parameter of the Weibull distribution.

    num_samples: int
        Number of times to run the scenario.

    Returns
    -------
    TTDs: list of dict
        Each dict stipulates the mean time to detection (and associated figures) for a particular scenario after
        being run num_samples amount of times. The confidence interval for time to detection is also included.
    """

    serial_interval_dist = weibull_min.rvs(shape, loc=0.5, scale=scale, size=num_samples)

    TTDs = []
    for scenario in scenarios:
        # Calculate TTD 'num_samples' amount of times
        sample_stats = []
        for sample in range(num_samples):
            sample_stats.append(
                TTD(**scenario, R=R, serial_interval=serial_interval_dist[sample])
            )

        # Calculate the mean of each attribute that has been calculated 'num_samples' amount of times
        median_stats = {}
        for key in list(sample_stats[0].keys()):
            median_stats[key] = statistics.median(
                [stats[key] for stats in sample_stats]
            )

        # Calculate confidence interval for TTD and total_cases
        times = [stats["TTD"] for stats in sample_stats]  
        TTD_confidence_interval = (np.percentile(times, 5), np.percentile(times, 95))

        cases = [stats["total_cases"] for stats in sample_stats]
        total_cases_confidence_interval = (
            np.percentile(cases, 5),
            np.percentile(cases, 95),
        )

        # Save summary/mean dictionary to scenario list
        TTDs.append(
            {
                "testing_regime": scenario["testing_regime"],
                "R_value": R,
                "TTD_confidence_interval": TTD_confidence_interval,
                "total_cases_confidence_interval": total_cases_confidence_interval,
                **median_stats,
            }
        )

    return TTDs


def summary_table(scenario_summary):
    """
    Takes a list of dictionaries that summarise an outbreak / testing scenario, reshapes / reformats them and saves
    a copy as .csv on the users desktop.

    Parameters
    ----------
    scenario_summary: list of dict
        Each dict stipulates the median time to detection (and associated figures) for a particular scenario after
        being run num_samples amount of times. The confidence interval for time to detection is also included.

    Returns
    -------
    df: pd.DataFrame
        Pandas dataframe with rounded/formatted figures from scenario_summary.
    """

    # Convert list of summary dictionaries to pandas dataframe
    df = pd.DataFrame(scenario_summary)

    # format TTD and confidence intervals to rounded integers
    df["TTD"] = df["TTD"].round()
    df["TTD_confidence_interval"] = df["TTD_confidence_interval"].apply(
        lambda x: (int(round(x[0], 0)), int(round(x[1], 0)))
    )
    df["total_cases_confidence_interval"] = df["total_cases_confidence_interval"].apply(
        lambda x: (int(round(x[0], 0)), int(round(x[1], 0)))
    )
    df = df.set_index(
        [
            "testing_regime",
            "R_value",
            "TTD_confidence_interval",
            "total_cases_confidence_interval",
            "TTD",
        ]
    )

    # If values is <=0.1, call it 0, if values >0.1 and <1, call it 1, else round to an integer
    for col in df.columns:
        df[col] = df[col].apply(
            lambda x: 0
            if x <= 0.1
            else ("<1" if (x > 0.1 and x < 1) else int(round(x)))
        )

    df = df.reset_index()

    # Get R value and format to use as string in saved file name
    R = '_'.join(str(df["R_value"][0]).split('.'))
    
    # Save table
    df.to_csv(f"TTD_{R}_{dt.date.today().strftime('%Y_%m_%d')}.csv", index=False)

    return df


def plot_scenarios(scenario_summary, variable="TTD"):
    """
    Takes a list of dictionaries that summarise an outbreak / testing scenario and plots the number of
    days before it is detected or the total number of cumulative cases in various testing scenarios.

    Parameters
    ----------
    scenario_summary: list of dict
        Each dict stipulates the median time to detection (and associated figures) for a particular scenario after
        being run num_samples amount of times. The confidence interval for time to detection is also included.
    variable: str (TDD or total_cases)
        Sets the variable to plot.
    """
    # Get attributes to plot
    labels = [scenario["testing_regime"] for scenario in scenario_summary]
    
    # Get R value
    R = [scenario["R_value"] for scenario in scenario_summary][0]
    
    # Format to use as string in saved file name
    R_string = '_'.join(str(R).split('.'))

    if variable == "TTD":
        variable_to_plot = [scenario["TTD"] for scenario in scenario_summary]
        errors = [scenario["TTD_confidence_interval"] for scenario in scenario_summary]
        y_label = "Days"
        title = f"Days to Detection when R = {R}"
        save_string = f"TTD_{R_string}_{dt.date.today().strftime('%Y_%m_%d')}.png"

    elif variable == "total_cases":
        variable_to_plot = [scenario["total_cases"] for scenario in scenario_summary]
        errors = [
            scenario["total_cases_confidence_interval"] for scenario in scenario_summary
        ]
        y_label = "No. infections"
        title = (
            f"Cumulative number of infections at point of first detection when R = {R}"
        )
        save_string = f"cumulative_cases_{R_string}_{dt.date.today().strftime('%Y_%m_%d')}.png"

    # Reformat confidence intervals
    lower = []
    upper = []
    for idx, i in enumerate(errors):
        lower.append(variable_to_plot[idx] - i[0])
        upper.append(i[1] - variable_to_plot[idx])
    errors = [lower, upper]

    # Make plot
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    plt.bar(labels, variable_to_plot, color="#21569E", alpha=0.9)
    plt.errorbar(
        labels,
        variable_to_plot,
        yerr=errors,
        fmt="none",
        ecolor="black",
        capsize=5,
        capthick=1,
    )
    plt.ylabel(y_label, fontsize=14)
    plt.title(title, fontsize=16)
    plt.xticks(range(len(labels)), labels, rotation=45, ha="right", fontsize=12)

    # Save plot
    plt.savefig(save_string, bbox_inches="tight", dpi=600)

### Determine population distribution of age bands in England

In [ ]:
# England age bands and populations
age_pop_dict = {"0-4": 3076945, "5-19": 9980931, "20-64": 33030917, "65+": 10401308}

# England age band populations as a proportion of total population
age_pop_distribution = {
    band: age_pop_dict[band] / sum(age_pop_dict.values()) for band in age_pop_dict
}

### Age specific IDRs, IHRs and IFRs (%) for community testing scenario 1
- For community testing scenarios, community testing is the detection mechanism. Therefore community testing scenarios have different rates for IDR and IHR.

In [ ]:
# scenario 1, community, untargeted
IDR_community_S1_untargeted = {"0-4": 0.1, "5-19": 0.1, "20-64": 0.1, "65+": 0.1}

IHR_community_S1_untargeted = {"0-4": 1, "5-19": 1, "20-64": 1, "65+": 1}

IFR_community_S1_untargeted = {"0-4": 0.0025, "5-19": 0.0025, "20-64": 0.0025, "65+": 0.0025}

# scenario 1, community, targeted
IDR_community_S1_targeted = {"0-4": 0.5, "5-19": 0.5, "20-64": 0.5, "65+": 0.5}

IHR_community_S1_targeted = {"0-4": 1, "5-19": 1, "20-64": 1, "65+": 1}

IFR_community_S1_targeted = {"0-4": 0.0025, "5-19": 0.0025, "20-64": 0.0025, "65+": 0.0025}

### Age specific IDRs and IFRs (%) for hospital testing scenarios 1, 2, and 3
- For hospital testing scenarios, hospitalisation is the detection mechanism, therefore the IDR and IHR rates are the same.
- ICU only testing scenarios takes the same IDRs, IHRs and IFRs as hospital testing scenarios, although the testing rate is different.

In [ ]:
# scenario 1, hospital, flat
IDR_hospital_S1_flat = {"0-4": 1, "5-19": 1, "20-64": 1, "65+": 1}

IFR_hospital_S1_flat = {"0-4": 0.25, "5-19": 0.25, "20-64": 0.25, "65+": 0.25}

# scenario 1, hospital, monotonic
IDR_hospital_S1_mono = {"0-4": 0.0041, "5-19": 0.0409, "20-64": 0.4091, "65+": 4.0913}

IFR_hospital_S1_mono = {"0-4": 0.001, "5-19": 0.0102, "20-64": 0.1023, "65+": 1.0228}

# scenario 1, hospital, young risk
IDR_hospital_S1_young = {"0-4": 2.1288, "5-19": 0.4258, "20-64": 0.04258, "65+": 4.2576}

IFR_hospital_S1_young = {"0-4": 0.5322, "5-19": 0.1064, "20-64": 0.0106, "65+": 1.0644}

# scenario 2, hospital, flat
IDR_hospital_S2_flat = {"0-4": 5, "5-19": 5, "20-64": 5, "65+": 5}

IFR_hospital_S2_flat = {"0-4": 2.5, "5-19": 2.5, "20-64": 2.5, "65+": 2.5}

# scenario 2, hospital, monotonic
IDR_hospital_S2_mono = {"0-4": 0.0205, "5-19": 0.2046, "20-64": 2.0457, "65+": 20.4566}

IFR_hospital_S2_mono = {"0-4": 0.0102, "5-19": 0.1023, "20-64": 1.0228, "65+": 10.2283}

# scenario 2, hospital, young risk
IDR_hospital_S2_young = {"0-4": 10.6439, "5-19": 2.1288, "20-64": 0.2129, "65+": 21.2878}

IFR_hospital_S2_young = {"0-4": 5.3220, "5-19": 1.0644, "20-64": 0.1064, "65+": 10.6439}

# scenario 3, hospital, flat
IDR_hospital_S3_flat = {"0-4": 10, "5-19": 10, "20-64": 10, "65+": 10}

IFR_hospital_S3_flat = {"0-4": 5, "5-19": 5, "20-64": 5, "65+": 5}

# scenario 3, hospital, monotonic
IDR_hospital_S3_mono = {"0-4": 0.0409, "5-19": 0.4091, "20-64": 4.0913, "65+": 40.9132}

IFR_hospital_S3_mono = {"0-4": 0.0205, "5-19": 0.2046, "20-64": 2.0457, "65+": 20.4566}

# scenario 3, hospital, young risk
IDR_hospital_S3_young = {"0-4": 21.2878, "5-19": 4.2576, "20-64": 0.4258, "65+": 42.5756}

IFR_hospital_S3_young = {"0-4": 10.6439, "5-19": 2.1288, "20-64": 0.2129, "65+": 21.2878}

### Scenarios

In [ ]:
scenarios = [
    # scenario 1, community, untargeted
    {
        "testing_regime": "Scenario 1 (Community Testing 1 in 1000)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_community_S1_untargeted,
        "age_specific_IHRs": IHR_community_S1_untargeted,
        "age_specific_IFRs": IFR_community_S1_untargeted,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 0,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },
    # scenario 1, community, targeted
    {
        "testing_regime": "Scenario 1 (Community Testing 1 in 200)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_community_S1_targeted,
        "age_specific_IHRs": IHR_community_S1_targeted,
        "age_specific_IFRs": IFR_community_S1_targeted,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 0,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },
    # scenario 1, hospital, flat
    {
        "testing_regime": "Scenario 1 (Hospital Testing - flat)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S1_flat,
        "age_specific_IHRs": IDR_hospital_S1_flat,
        "age_specific_IFRs": IFR_hospital_S1_flat,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },
    # scenario 1, hospital, monotonic
    {
        "testing_regime": "Scenario 1 (Hospital Testing - monotonic)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S1_mono,
        "age_specific_IHRs": IDR_hospital_S1_mono,
        "age_specific_IFRs": IFR_hospital_S1_mono,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },
    # scenario 1, hospital, young risk
    {
        "testing_regime": "Scenario 1 (Hospital Testing - young risk)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S1_young,
        "age_specific_IHRs": IDR_hospital_S1_young,
        "age_specific_IFRs": IFR_hospital_S1_young,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },
    # scenario 1, ICU, flat
    {
        "testing_regime": "Scenario 1 (ICU Testing - flat)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S1_flat,
        "age_specific_IHRs": IDR_hospital_S1_flat,
        "age_specific_IFRs": IFR_hospital_S1_flat,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 0.25,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 2,
        "testing_delay": 2,
    },
    # scenario 1, ICU, monotonic
    {
        "testing_regime": "Scenario 1 (ICU Testing - monotonic)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S1_mono,
        "age_specific_IHRs": IDR_hospital_S1_mono,
        "age_specific_IFRs": IFR_hospital_S1_mono,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 0.25,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 2,
        "testing_delay": 2,
    },
    # scenario 1, ICU, young risk
    {
        "testing_regime": "Scenario 1 (ICU Testing - young risk)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S1_young,
        "age_specific_IHRs": IDR_hospital_S1_young,
        "age_specific_IFRs": IFR_hospital_S1_young,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 0.25,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 2,
        "testing_delay": 2,
    },
    # scenario 2, hospital, flat
    {
        "testing_regime": "Scenario 2 (Hospital Testing - flat)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S2_flat,
        "age_specific_IHRs": IDR_hospital_S2_flat,
        "age_specific_IFRs": IFR_hospital_S2_flat,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },
    # scenario 2, hospital, monotonic
    {
        "testing_regime": "Scenario 2 (Hospital Testing - monotonic)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S2_mono,
        "age_specific_IHRs": IDR_hospital_S2_mono,
        "age_specific_IFRs": IFR_hospital_S2_mono,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },
    # scenario 2, hospital, young risk
    {
        "testing_regime": "Scenario 2 (Hospital Testing - young risk)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S2_young,
        "age_specific_IHRs": IDR_hospital_S2_young,
        "age_specific_IFRs": IFR_hospital_S2_young,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },
    # scenario 2, ICU, flat
    {
        "testing_regime": "Scenario 2 (ICU Testing - flat)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S2_flat,
        "age_specific_IHRs": IDR_hospital_S2_flat,
        "age_specific_IFRs": IFR_hospital_S2_flat,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 0.25,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 2,
        "testing_delay": 2,
    },
    # scenario 2, ICU, monotonic
    {
        "testing_regime": "Scenario 2 (ICU Testing - monotonic)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S2_mono,
        "age_specific_IHRs": IDR_hospital_S2_mono,
        "age_specific_IFRs": IFR_hospital_S2_mono,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 0.25,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 2,
        "testing_delay": 2,
    },
    # scenario 2, ICU, young risk
    {
        "testing_regime": "Scenario 2 (ICU Testing - young risk)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S2_young,
        "age_specific_IHRs": IDR_hospital_S2_young,
        "age_specific_IFRs": IFR_hospital_S2_young,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 0.25,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 2,
        "testing_delay": 2,
    },
    # scenario 3, hospital, flat
    {
        "testing_regime": "Scenario 3 (Hospital Testing - flat)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S3_flat,
        "age_specific_IHRs": IDR_hospital_S3_flat,
        "age_specific_IFRs": IFR_hospital_S3_flat,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },
    # scenario 3, hospital, monotonic
    {
        "testing_regime": "Scenario 3 (Hospital Testing - monotonic)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S3_mono,
        "age_specific_IHRs": IDR_hospital_S3_mono,
        "age_specific_IFRs": IFR_hospital_S3_mono,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },    
    # scenario 3, hospital, young risk
    {
        "testing_regime": "Scenario 3 (Hospital Testing - young risk)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S3_young,
        "age_specific_IHRs": IDR_hospital_S3_young,
        "age_specific_IFRs": IFR_hospital_S3_young,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 1,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 0,
        "testing_delay": 2,
    },
    # scenario 3, ICU, flat
    {
        "testing_regime": "Scenario 3 (ICU Testing - flat)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S3_flat,
        "age_specific_IHRs": IDR_hospital_S3_flat,
        "age_specific_IFRs": IFR_hospital_S3_flat,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 0.25,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 2,
        "testing_delay": 2,
    },
    # scenario 3, ICU, monotonic
    {
        "testing_regime": "Scenario 3 (ICU Testing - monotonic)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S3_mono,
        "age_specific_IHRs": IDR_hospital_S3_mono,
        "age_specific_IFRs": IFR_hospital_S3_mono,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 0.25,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 2,
        "testing_delay": 2,
    },    
    # scenario 3, ICU, young risk
    {
        "testing_regime": "Scenario 3 (ICU Testing - young risk)",
        "initial_cases": 1,
        "age_specific_IDRs": IDR_hospital_S3_young,
        "age_specific_IHRs": IDR_hospital_S3_young,
        "age_specific_IFRs": IFR_hospital_S3_young,
        "age_pop_distribution": age_pop_distribution,
        "testing_rate": 0.25,
        "hospital_admission_delay": 9,
        "ICU_admission_delay": 2,
        "testing_delay": 2,
    },
]

In [ ]:
# Map of full scenario names and their abbreviated version
scenario_name_short = {
    "Scenario 1 (Community Testing 1 in 1000)":"Scenario 1 (Community Testing 1 in 1000)",
    "Scenario 1 (Community Testing 1 in 200)":"Scenario 1 (Community Testing 1 in 200)",
    "Scenario 1 (Hospital Testing - flat)":"Scenario 1 (Hospital Testing)",
    "Scenario 1 (Hospital Testing - monotonic)":"Scenario 1 (Hospital Testing)",
    "Scenario 1 (Hospital Testing - young risk)":"Scenario 1 (Hospital Testing)",
    "Scenario 1 (ICU Testing - flat)":"Scenario 1 (ICU Testing)",
    "Scenario 1 (ICU Testing - monotonic)":"Scenario 1 (ICU Testing)",
    "Scenario 1 (ICU Testing - young risk)":"Scenario 1 (ICU Testing)",
    "Scenario 2 (Hospital Testing - flat)":"Scenario 2 (Hospital Testing)",
    "Scenario 2 (Hospital Testing - monotonic)":"Scenario 2 (Hospital Testing)",
    "Scenario 2 (Hospital Testing - young risk)":"Scenario 2 (Hospital Testing)",
    "Scenario 2 (ICU Testing - flat)":"Scenario 2 (ICU Testing)",
    "Scenario 2 (ICU Testing - monotonic)":"Scenario 2 (ICU Testing)",
    "Scenario 2 (ICU Testing - young risk)":"Scenario 2 (ICU Testing)",
    "Scenario 3 (Hospital Testing - flat)":"Scenario 3 (Hospital Testing)",
    "Scenario 3 (Hospital Testing - monotonic)":"Scenario 3 (Hospital Testing)",
    "Scenario 3 (Hospital Testing - young risk)":"Scenario 3 (Hospital Testing)",
    "Scenario 3 (ICU Testing - flat)":"Scenario 3 (ICU Testing)",
    "Scenario 3 (ICU Testing - monotonic)":"Scenario 3 (ICU Testing)",
    "Scenario 3 (ICU Testing - young risk)":"Scenario 3 (ICU Testing)",
}

# Cut and paste from this list to 'scenario_filter' to stipulate which scenarios to plot.
list_of_scenarios = [scenario["testing_regime"] for scenario in scenarios]
list_of_scenarios

In [ ]:
# List of the scenarios to be plotted
scenario_filter = [
    "Scenario 1 (Community Testing 1 in 1000)",
    "Scenario 1 (Community Testing 1 in 200)",
    "Scenario 1 (Hospital Testing - flat)",
    "Scenario 1 (ICU Testing - flat)",
    "Scenario 2 (Hospital Testing - flat)",
    "Scenario 2 (ICU Testing - flat)",
    "Scenario 3 (Hospital Testing - flat)",
    "Scenario 3 (ICU Testing - flat)",
]

### Days to detection when R = 1.2

In [ ]:
scenario_summary = assess_scenarios(scenarios, R=1.2, num_samples=100000)

In [ ]:
summary_table(scenario_summary)

In [ ]:
# Select scenarios you wish to plot
scenario_summary = [
    scenario
    for scenario in scenario_summary
    if scenario["testing_regime"] in scenario_filter
]

# Amend labels to their abbreviated version if desired
scenario_summary = [
    {
        key: (scenario_name_short[scenario[key]] if key == "testing_regime" else value)
        for key, value in scenario.items()
    }
    for scenario in scenario_summary
]

In [ ]:
plot_scenarios(scenario_summary, variable="TTD")

### Days to detection when R = 2.0

In [ ]:
scenario_summary = assess_scenarios(scenarios, R=2.0, num_samples=100000)

In [ ]:
summary_table(scenario_summary)

In [ ]:
# Select scenarios you wish to plot
scenario_summary = [
    scenario
    for scenario in scenario_summary
    if scenario["testing_regime"] in scenario_filter
]

# Amend labels if desired
scenario_summary = [
    {
        key: (scenario_name_short[scenario[key]] if key == "testing_regime" else value)
        for key, value in scenario.items()
    }
    for scenario in scenario_summary
]

In [ ]:
plot_scenarios(scenario_summary, variable="TTD")